
## 0. Install dependencies


In [1]:
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "-q"])
print("Installation complete")

✅ Installation complete


In [2]:
# Configure environment
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
print(f"API Key: {os.getenv('GOOGLE_API_KEY', '')[:10]}...")

✅ API Key: AIzaSyBC8Z...


## 0.1 Configure Environment Variables

**Set up your Google API key and other configuration:**

# MomsHelperAI - Multi-Agent System Demo

**Google ADK Multi-Agent System for Family Planning**

This notebook demonstrates a production-ready multi-agent system using:
- Google Agent Development Kit (ADK)
- Gemini 2.5 Flash Lite LLM
- SQLite for data persistence

## 1. Setup and Imports

In [3]:
import asyncio
import os
from datetime import datetime, timedelta
import importlib
import sys

# Force reload modules to get latest changes
if 'agents.meal_planner' in sys.modules:
    importlib.reload(sys.modules['agents.base_agent'])
    importlib.reload(sys.modules['agents.search_agent'])
    importlib.reload(sys.modules['agents.meal_planner'])
    importlib.reload(sys.modules['agents.week_planner'])
    importlib.reload(sys.modules['agents.grocery_planner'])
    importlib.reload(sys.modules['agents.orchestrator'])
    print("Reloaded agent modules")

# MomsHelperAI imports
from agents.orchestrator import orchestrator
from agents.meal_planner import meal_planner_agent
from agents.week_planner import week_planner_agent
from agents.grocery_planner import grocery_planner_agent
from storage.sqlite_storage import SQLiteStorage
from utils.config import Config

print("All imports successful")
print(f"API Key configured: {Config.GOOGLE_API_KEY[:10]}...")
print(f"Using Gemini model: gemini-2.5-flash-lite")

2025-12-01 00:12:57 - agents.base_agent - INFO - SearchAgent initialized
2025-12-01 00:12:57 - agents.search_agent - INFO - SearchAgent initialized with google_search tool
2025-12-01 00:12:57 - agents.search_agent - INFO - SearchAgent initialized with google_search tool
2025-12-01 00:12:57 - agents.base_agent - INFO - MealPlannerAgent initialized
2025-12-01 00:12:57 - agents.meal_planner - INFO - MealPlannerAgent initialized
2025-12-01 00:12:57 - agents.base_agent - INFO - MealPlannerAgent initialized
2025-12-01 00:12:57 - agents.meal_planner - INFO - MealPlannerAgent initialized
2025-12-01 00:12:57 - agents.base_agent - INFO - WeekPlannerAgent initialized
2025-12-01 00:12:57 - agents.week_planner - INFO - WeekPlannerAgent initialized
2025-12-01 00:12:57 - agents.base_agent - INFO - WeekPlannerAgent initialized
2025-12-01 00:12:57 - agents.week_planner - INFO - WeekPlannerAgent initialized
2025-12-01 00:12:57 - agents.base_agent - INFO - GroceryPlannerAgent initialized
2025-12-01 00:12

## 2. Initialize Storage and Load Sample Data

In [4]:
# Initialize storage
storage = SQLiteStorage()

# Load sample Sharma family
sharma_family = {
    'id': 'sharma_001',
    'name': 'Sharma Family',
    'members': [
        {'name': 'Rajesh', 'age': 38, 'role': 'father'},
        {'name': 'Priya', 'age': 35, 'role': 'mother'},
        {'name': 'Aarav', 'age': 10, 'role': 'son'},
        {'name': 'Ananya', 'age': 7, 'role': 'daughter'}
    ],
    'dietary_restrictions': ['vegetarian'],
    'preferred_cuisines': ['North Indian', 'South Indian', 'Gujarati'],
    'allergies': [],
    'spice_level': 'medium'
}

try:
    storage.create_family(sharma_family)
    print("Sharma family data loaded")
except:
    print("Sharma family already exists")

print(f"\nFamily: {sharma_family['name']}")
print(f"   Members: {len(sharma_family['members'])} (Rajesh, Priya, Aarav, Ananya)")
print(f"   Dietary: Vegetarian")
print(f"   Cuisines: {', '.join(sharma_family['preferred_cuisines'])}")

2025-12-01 00:13:06 - momshelper - INFO - SQLite database initialized at ./data/momshelper.db
2025-12-01 00:13:06 - momshelper - INFO - Saved family profile for sharma_001
2025-12-01 00:13:06 - momshelper - INFO - Saved family profile for sharma_001
✅ Sharma family data loaded

👨‍👩‍👧‍👦 Family: Sharma Family
   Members: 4 (Rajesh, Priya, Aarav, Ananya)
   Dietary: Vegetarian
   Cuisines: North Indian, South Indian, Gujarati
✅ Sharma family data loaded

👨‍👩‍👧‍👦 Family: Sharma Family
   Members: 4 (Rajesh, Priya, Aarav, Ananya)
   Dietary: Vegetarian
   Cuisines: North Indian, South Indian, Gujarati


## 3. Demo 1: Meal Planning with MealPlannerAgent

**This demonstrates:**
- MealPlannerAgent using google_search for web recipes
- Real LLM response from Gemini

### Important: API Rate Limits

**Free tier limits for `gemini-2.5-flash-lite`:**
- 15 requests per minute
- Each meal plan uses ~5-10 API calls
- **Wait 60 seconds between runs if you get quota errors**

If you see "quota exceeded" error:
1. Wait 1 minute
2. Re-run the cell
3. Or use a different API key

In [5]:
print("Planning meals for the Sharma family...\n")
print("Calling MealPlanner Agent...")

try:
    response = await meal_planner_agent.plan_meals(
        family_id="sharma_001",
        request="Suggest easy and fast recepi for monday and tuesday, keep break on wednesday, thursday if not activity and have time then heavy meal.",
        num_days=7
    )
    
    print("="*70)
    print("MEAL PLAN RESPONSE:")
    print("="*70)
    
    # Extract the final text response from events
    if isinstance(response, list):
        # Find the last event with text content
        found_text = False
        for event in reversed(response):
            if hasattr(event, 'content') and event.content:
                # Check if parts exists and is not None
                if hasattr(event.content, 'parts') and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            print(part.text)
                            found_text = True
                            break
                    if found_text:
                        break
        
        if not found_text:
            print("No text response found in events")
            print(f"\nTotal events: {len(response)}")
            # Show last few events for debugging
            if response:
                print("\nLast event details:")
                last_event = response[-1]
                print(f"  Event type: {type(last_event)}")
                if hasattr(last_event, 'content'):
                    print(f"  Has content: {last_event.content is not None}")
                    if last_event.content and hasattr(last_event.content, 'parts'):
                        print(f"  Parts: {last_event.content.parts}")
    elif hasattr(response, 'text'):
        print(response.text)
    else:
        print(response)
    
    print("\n" + "="*70)
    print("Real AI-generated meal plan from Gemini")
    print("="*70)
    
except Exception as e:
    error_msg = str(e)
    print(f"Error message: {error_msg}")

🍽️ Planning meals for the Sharma family...

⏳ Calling MealPlanner Agent...

 ### Created new session: debug_session_id

User > Plan meals for family sharma_001.

User request: Suggest easy and fast recepi for monday and tuesday, keep break on wednesday, thursday if not activity and have time then heavy meal.
Days: 7
Dietary restrictions: none
Preferences: Indian cuisine

1. Use get_family_preferences tool
2. Use SearchAgent to find recipes
3. Create meal plan with the recipes found
4. Save using save_meal_plan tool

Return complete meal plan with recipes.


2025-12-01 00:13:26 - momshelper - INFO - SQLite database initialized at ./data/momshelper.db
MealPlannerAgent > {"meal_plan": [{"day": "Monday", "breakfast": {"meal_name": "Poha", "prep_time_minutes": 15, "servings": 4, "ingredients": ["poha", "onion", "mustard seeds", "curry leaves", "turmeric", "salt", "lemon juice", "peanuts", "cilantro"], "recipe_steps": "1. Rinse poha (flattened rice) and set aside. 2. Heat oil in a pan, add mustard seeds and curry leaves. 3. Add chopped onions and sauté until translucent. 4. Add turmeric, salt, and peanuts. Sauté for a minute. 5. Add the rinsed poha and mix well. 6. Cook for 5-7 minutes, stirring occasionally. 7. Garnish with fresh cilantro and lemon juice.", "reference_link": "https://www.vegrecipesofindia.com/poha-recipe-maharashtrian-kande-pohe/"}, "lunch": {"meal_name": "Dal Fry with Rice", "prep_time_minutes": 25, "servings": 4, "ingredients": ["toor dal", "onion", "tomato", "ginger-garlic paste", "turmeric powder", "cumin seeds", "mustard 

## 4. Demo 2: Natural Language Conversation

**Free-form chat with the orchestrator - Try your own request!**

In [6]:
# EDIT THIS - Try your own request
my_request = "I need quick dinner ideas for tonight that kids will like"

print(f"Your request: {my_request}\n")
print("Calling MomsHelperAI orchestrator...\n")

response = await orchestrator.handle_request(
    user_request=my_request,
    family_id='sharma_001'
)

print("="*70)
print("AI Response:")
print("="*70)

if hasattr(response, 'text'):
    print(response.text)
else:
    print(response)

print("\nReal-time AI response using Google ADK and Gemini")

Your request: I need quick dinner ideas for tonight that kids will like

⏳ Calling MomsHelperAI orchestrator...

2025-12-01 00:14:59 - agents.orchestrator - INFO - Orchestrating request: I need quick dinner ideas for tonight that kids will like...
2025-12-01 00:14:59 - agents.orchestrator - INFO - Step 1: Calling MealPlannerAgent...
2025-12-01 00:14:59 - agents.orchestrator - INFO - Step 1: Calling MealPlannerAgent...

 ### Continue session: debug_session_id

User > Plan meals for family sharma_001.

User request: I need quick dinner ideas for tonight that kids will like
Days: 7
Dietary restrictions: none
Preferences: Indian cuisine

1. Use get_family_preferences tool
2. Use SearchAgent to find recipes
3. Create meal plan with the recipes found
4. Save using save_meal_plan tool

Return complete meal plan with recipes.

 ### Continue session: debug_session_id

User > Plan meals for family sharma_001.

User request: I need quick dinner ideas for tonight that kids will like
Days: 7
Dietar

==============================================================================

## Conclusion

This notebook demonstrated **MomsHelperAI** - a production-ready multi-agent system using:

- **Google Agent Development Kit (ADK)**
- **Gemini 2.5 Flash Lite LLM** (Real AI responses)
- **Multi-agent architecture** with specialized sub-agents
- **SQLite** for data persistence
- **Family context** - culturally aware meal planning and activities

==============================================================================

**Built using Google Agent Development Kit (ADK)**